In [1]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import os
os.environ["PATH"] += os.pathsep +  r"C:\Users\NILESH\Anaconda3\Library\bin\graphviz"
from sklearn.tree import export_graphviz
import pydotplus
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
titanic = pd.read_csv(r"C:\Users\NILESH\Downloads\titanic_train.csv")
df = titanic.copy()

In [25]:
df.head()

,Parch,Pclass,Sex,Age,SibSp,Embarked,Survived
0,0,3,0,22.0,1,1,0
1,0,1,1,38.0,1,0,1
2,0,3,1,26.0,0,1,1
3,0,1,1,35.0,1,1,1
4,0,3,0,35.0,0,1,0


In [4]:
del df["PassengerId"]
df.drop('Cabin' , inplace  = True , axis = 1)
del df["Ticket"]
del df["Fare"]

In [5]:
del df['Name']

In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [7]:
def Embarked_num(s):
    if s == "C":
        return 0
    elif s == "S":
        return 1
    elif s == "Q":
        return 2
    else:
        return -1
df['Embarked'] = df.Embarked.apply(Embarked_num)

In [8]:
a = df[df.Survived == 1].Age.sum()
b = df[df.Survived == 1].Age.sum()
df.loc[(df.Survived == 1) & (df.Age.isnull()) , 'Age'] = a
df.loc[(df.Survived == 0) & (df.Age.isnull()) , 'Age'] = b
df.loc[(df.Survived == 1) & (df.Embarked.isnull()) , 'Embarked'] = df[df.Survived == 1].Embarked.mode()
df.loc[(df.Survived == 0) & (df.Embarked.isnull()) , 'Embarked'] = df[df.Survived == 0].Embarked.mode()
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [9]:
def gender_num(s):
    if s == "male":
        return 0
    return 1
df['Sex'] = df.Sex.apply(gender_num)

In [10]:
cols = df.columns.tolist()
cols[0] , cols[5] = cols[5] , cols[0]
cols[5] , cols[6] = cols[6] , cols[5]
df = df[cols]
df.head()

,Parch,Pclass,Sex,Age,SibSp,Embarked,Survived
0,0,3,0,22.0,1,1,0
1,0,1,1,38.0,1,0,1
2,0,3,1,26.0,0,1,1
3,0,1,1,35.0,1,1,1
4,0,3,0,35.0,0,1,0


In [26]:
X = df.iloc[: , 0 : df.shape[1] - 1]
Y = df.iloc[: , - 1]

In [27]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y)

In [41]:
alg1 = RandomForestClassifier(random_state = 0 , criterion = 'entropy' , n_estimators = 15)
alg1.fit(X_test , Y_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [42]:
Y_predicted = alg1.predict(X_test)
Y_pred_train = alg1.predict(X_train)
alg1.score(X_train , Y_train) , alg1.score(X_test , Y_test)

(0.7664670658682635, 0.9506726457399103)

In [24]:
dot_data = export_graphviz(alg1.estimators_[0] , out_file = None , rounded = True , filled = True )
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("TREE.pdf")